**Overview of NLP (L90) Practical Session 1**

Welcome to the first practical accompanying the Overview of NLP (L90) lecture course. Overall, the purpose of the practicals is to build and evaluate an NLP system. Specifically, we are going to work on **named entity recognition**.

In this session we will introduce a 3-part practical task.

1.   Explore and annotate a named entity recognition dataset (today, due 27 October 3pm, worth 10%)
2.   Attempt feature-based NER (next week, due 10 November 3pm, worth 10%)
3.   Attempt NER with neural networks and write a report (11 November, due 3 December 3pm, worth 80%)

Note that all submissions are made via the [course Moodle page](https://www.vle.cam.ac.uk/course/view.php?id=206751), and that you can only view this Colab notebook (including the ability to run code blocks) not edit it: you can make a copy of your own in the File menu if you do want to edit anything, but there is no need to.

Ok, let's start looking at the task.

**Named Entity Recognition: W-NUT 2017 Shared Task**

In 2017, the annual Workshop on Noisy User-generated Text (W-NUT) set a shared task (competition) on 'novel and emerging named entity recognition'.

_Named entity recognition_ (NER) is an information extraction task which involves automatic identification of entities of interest: e.g. people's names, organisations, locations, dates, and so on.

```
This year's {Tour de France}_MISC began in Brittany_LOC on {26 June 2021}_DATE and was won by {Tadej Pogačar}_PERS of Slovenia_LOC.
```

It has long been a feature of shared tasks, which are open research competitions. Notably, the CoNLL^ 2002 and 2003 shared tasks involved multilingual NER, but the text type for these tasks was newswire, and consequently the named entity types were fairly standard, well-formed and highly predictable. This meant that leading systems for the shared tasks performed very well, and NER was perhaps seen by some as a 'solved problem', in the sense that machine models could do almost as well as humans -- for instance correctly identifying about 9 in 10 entities in the English test set (remember that humans make mistakes and don't always agree on categorisation).

^"[The SIGNLL Conference on Computational Natural Language Learning](https://conll.org/previous-editions)", a prestigious annual conference running since the late-1990s.

However, named entities are guaranteed to change, and not everyone produces named entities in well-formed, fully-formed or consistent ways. There has consequently been a growing interest in NER on user-generated texts: blogposts, social media, consumer reviews, SMS, interaction with chatbots and home assistants, etc. Standard NER systems were not found to work very well on these.

Thus, _novel_, _emerging_ named entities remain a challenging and interesting detection task. 'Novel' or 'emerging' entities may be those we have not encountered before (e.g. COVID-19 before 2019) or ones we have encountered before but appearing in non-standard form (e.g. 'Rona' as a way to refer to coronavirus).

In 2017, W-NUT organised a shared task on novel, emerging NER and the [competition site](https://noisy-text.github.io/2017/emerging-rare-entities.html) is still online.


**The data**

The shared task dataset is split into training, development and test sections. The training data comprises 3.4K tweets (62.5K word tokens), the development set contains YouTube comments on popular videos (1K docs, 15.6K tokens), the test set contains a mix of other tweets, Reddit posts, and StackExchange comments (1.3K docs, 23.3K tokens).

Note that the texts have not been screened for obscene language, though we have filtered some texts containing offensive words (hence corpus size differences with those described in the [shared task overview paper](https://aclanthology.org/W17-4418/)). Please alert us if you find anything in the data which is offensive! We think it's important to work with authentic datasets in an objective fashion but at the same time do not wish to disseminate anything which offends.

Any texts containing named entities found in the training set were removed from the development and test sets by the task organisers, ensuring that entities in those sets were not encountered in training. All texts have already been tokenized and are presented in tab-delimited 'CoNLL' format, with one token per line, an empty line between texts, and two columns: firstly the tokens, secondly the named entity labels.

For instance, in the text below we see the BIO annotation scheme in use (beginning, inside, outside). 'Empire' is marked as the beginning of a location entity, subsequent 'State' and 'Building' tokens are marked as part of that same entity through the use of inside labels. An outside label for '=' marks the end of that first entity, and it is followed by a second B-location label for 'ESB'.

| token | label |
| --- | --- |
| Empire | B-location |
| State | I-location |
| Building | I-location |
| = | O |
| ESB | B-location |
| . | O |

**The task**

The shared task required that entrants predict the named entity labels for the test set, based on the training and development sets only (i.e. no external data). Conventional information retrieval metrics -- precision, recall and F-measure -- were used to rank submissions, but more on this in session 2. For now, let's look at the data some more.

**Inspecting the data**

_pandas_ is a popular library for data handling in Python. Let's use it to load the shared task training file (from Google Cloud Storage) and inspect the first 10 rows. Note that the normal format for tokens and labels is tab-delimited plain-text tables, so-called 'CoNLL' format (which has various versions based on how many columns / how much information is included). There's one word token per row (note that the text has been tokenised, see "It" and "'s") and a named entity label in the 2nd column of 'BIO' type where 'B' = begin, 'I' = inside, 'O' = outside. This means that a new named entity is marked with a B, a continued multi-token named entity is marked by I, and all other tokens are marked O for 'outside'.

In [ ]:
import pandas as pd
wnuttrain = 'https://storage.googleapis.com/wnut-2017_ner-shared-task/wnut17train.txt'
train = pd.read_table(wnuttrain, header=None, names=['token', 'label'])
train.head(n=10)

,token,label
0,@paulwalk,O
1,It,O
2,'s,O
3,the,O
4,view,O
5,from,O
6,where,O
7,I,O
8,'m,O
9,living,O


It's a little surprising that the "@paulwalk" username in row 0 is not tagged as an entity. So let's look more closely at the annotation guidelines for this dataset.

**Annotation**

Firstly, note that the training set was annotated as part of a different project prior to the shared task in 2017 -- so we need to look back a bit further to the W-NUT shared task 2015 ([Baldwin et al 2015](https://www.aclweb.org/anthology/W15-4319)) when the Twitter dataset had been previously used for NER. These tweets had been collected in 2010 and annotated for 10 different entity types: person, location, company, facility, product, band, sports team, movie, TV show, other ([Ritter et al 2011](https://www.aclweb.org/anthology/D11-1141)). Later these classes were reduced to 6 as described below.

Back to the 2017 shared task ([Derczynski et al 2017](https://www.aclweb.org/anthology/W17-4418)) and we find that the dev and test sets were annotated afresh in preparation for the shared task. The texts were shown to workers on the CrowdFlower crowdsourcing platform, and each text was annotated by 3 different workers before a final review was made by the shared task organisers.

Annotators were asked to label 6 entity types:

1. person (referred to by name; real and fictional)
2. location (includes facility; real and fictional)
3. corporation
4. product (real and fictional, official names only)
5. creative-work (should be human-made; e.g. song titles)
6. group (includes music bands, sports teams, non-corporate organisations)

Perhaps you can already see why some of these may be difficult to label with any certainty in user-generated text? We also see why the Twitter username was not labelled in the example above: because the person was not referred to by their name.

Annotators were required to be from a certain number of English-speaking countries (though note that this does not entail that they are all native speakers of English). Annotations were initially processed with max-recall automatic adjudication: "Any time a span is annotated by a worker, that span is placed in the final set. Adjacent annotations of the same type are concatenated" ([Derczynski et al 2016](https://www.aclweb.org/anthology/C16-1111)). Then the task organisers performed a manual review. Remember as well that texts featuring any entities found in the training set were removed from the dev and test sets.

For more information about how the entities types were selected, see the [shared task README file](https://storage.googleapis.com/wnut-2017_ner-shared-task/wnut17_README.md).

The following numbers of entities were found in the dev and test sets:

|  | Train | Dev | Test |
| --- | --- | --- | --- |
| Documents | 3,372 | 993 | 1,283 |
| Tokens | 62,241 | 15,382 | 23,323 |
| Entities | 1,975 | 836 | 1,079 |
| --- | --- | --- | --- |
| person | 660 | 470 | 429 |
| location | 548 | 74 | 150 |
| corporation | 221 | 34 | 66 |
| product | 142 | 114 | 127 |
| creative-work | 140 | 105 | 142 |
| group | 264 | 39 | 165 |


**Assignment 1**

Your first assignment is to annotate some of the texts yourselves, in order to understand the task and the entity types. 

We've randomly selected 25 texts from the training set which were not tagged with any named entities and 25 texts which did contain named entities, according to the official annotation. Even the official annotation will be open to debate, because there's some degree of subjectivity in any annotation task, and therefore this assignment does not require that you do exactly the same annotation as the original file. 

**The requirements for a tick in this assignment are to submit a correctly formatted text file in which you have made some edits** (you could submit an unedited text file, if you are sure that there are no named entities in the texts, but we would find this odd and ask you to confirm that you have closely read all the texts).

Your starting file for this assignment may be found [here](https://storage.googleapis.com/wnut-2017_ner-shared-task/wnut17train_sample_blank.txt). It contains the 50 texts in tabular format, with one word token per line, and a blank line between texts. Word tokens are in the first column, and named entity labels are in the second column. There are no column headers. The file is blank in that all named entity labels are set to 'O' (other). Your task is to update these with 'B' (beginning) or 'I' (inside) labels where you can identify named entities. This is the so-called BIO tagging scheme: 'B' marks the start of a named entity, 'I' indicates continuation of that same named entity (i.e. it is more than one word token), and the 'O' label is for anything else.

Remember that there are 6 possible entity types: person, location, corporation, product, creative work, group. Therefore the possible values for the labels column are 'B-person', 'B-location', 'B-corporation', 'B-product', 'B-creative-work', 'B-group', 'I-person', 'I-location', 'I-corporation', 'I-product', 'I-creative-work', 'I-group', and 'O'.

You could edit the file using a dedicated annotation tool such as [BRAT](https://brat.nlplab.org), in a text editor such as `vim`, `nano` or `emacs`, or as a spreadsheet. The important thing is to save your annotated file as plain text maintaining the tab-delimited format, with blank lines between texts and no column headers. **Failure to do so will result in no tick being awarded: please check your file format by reading it with _pandas_ before submission.**

Please replace 'blank' in the filename with your CRSid and upload your annotated file using the correct link on [Moodle](https://www.vle.cam.ac.uk/course/view.php?id=206751). We will let you know your agreement kappa with the original file (though remember that this is not a method of assessment: you get a tick for submitting a correctly-formatted, edited file). One final comment: clearly you could obtain the original training file and search for the 'correct' annotations, but where would the fun be in that? (and also, it'll be pretty noticeable if you get absolute agreement of 100%!)